In [6]:
import pandas as pd
import pymysql
import numpy as np
import datetime
import os

#establish db connection
con = pymysql.connect('localhost', 'root', 'HIDDEN', 'HIDDEN')

<strong>This file is for merging all CSV files from Ilmatieteenlaitos.fi, containing detailed hourly weather station data from Vantaa airport. It then inserts the values into MySQL.</strong>

In [2]:
def merge_csv(files):
    '''
    Takes in list of file names in the immediate directory, filters them if .csv file, then returns a DataFrame.
    
    Parameters: 
        list: all values are (str) from immediate directory.
    
    Returns:
        Pandas DataFrame containing:
        Int64Index: 106656 entries, 97897 to 19007
        Data columns (total 12 columns):
         #   Column      Non-Null Count   Dtype         
        ---  ------      --------------   -----         
         0   timestamp   106656 non-null  datetime64[ns]
         1   temp        106656 non-null  float64       
         2   dew point   106656 non-null  float64       
         3   humidity    106656 non-null  float64       
         4   precip mm   106656 non-null  float64       
         5   msl         106656 non-null  float64       
         6   clouds      106656 non-null  float64       
         7   visibility  106656 non-null  float64       
         8   wind speed  106656 non-null  float64       
         9   wind gust   106656 non-null  float64       
         10  wind dir    106656 non-null  float64       
         11  snow        106656 non-null  int64         
        dtypes: datetime64[ns](1), float64(10), int64(1)
    
    '''
    #pass file list to variable, filter if .csv. merge all file data together. sort index by date values.
    files = os.listdir('.')
    file_list = []

    for file in files:
        if 'csv' in file:
            name = file[:9] 
            name = pd.read_csv(file)
            file_list.append(name)
            
    merged = pd.concat(file_list)
    merged = merged.reset_index(drop=True).sort_values(['Vuosi','Kk','Pv'])
    
    #combine date values to create datetime index, drop unneccesary columns
    merged['timestamp'] = pd.to_datetime(merged[merged.columns[0]].astype(str) +'-'+ merged[merged.columns[1]].astype(str)  +'-'+ merged[merged.columns[2]].astype(str) +' '+ merged[merged.columns[3]].astype(str))
    merged = merged.drop(columns=list(merged.columns[:5]))
    merged = merged.drop(merged.columns[4], axis=1)
    
    #rename columns
    merged = merged.rename(columns={merged.columns[0]:'clouds',merged.columns[1]:'msl',merged.columns[2]:'precip mm',merged.columns[3]:'humidity',merged.columns[4]:'snow',merged.columns[5]:'temp',merged.columns[6]:'dew point',merged.columns[7]:'visibility',merged.columns[8]:'wind dir',merged.columns[9]:'wind gust',merged.columns[10]:'wind speed'})

    #some rows contain strings or a dash, filter those and reassign integer instead
    merged.loc[merged['snow'].str.contains("^-$",na=False,regex=True),'snow'] = 0
    merged['snow'] = merged['snow'].fillna(0).astype(int)
    merged.loc[merged['snow'] < 0,'snow']= 0
    
    #cloudiness is determined as an integer 1/8. some values are over 8 (fog), so reassign those values to be 8 instead.
    merged.loc[merged['clouds'] > 8,'clouds'] = 8
    
    #some rows contain strings or a dash, filter those and reassign integer instead
    merged.loc[merged['precip mm'].str.contains("^-$",na=False,regex=True),'precip mm']  = 0
    merged['precip mm'] = merged['precip mm'].astype(float)
    merged.loc[merged['precip mm'] < 0,'precip mm']= 0
    
    #get percentage values of cloudiness instead of 1/8
    merged['clouds'] = merged['clouds'] * 12.5
    
    #drop any duplicates, if any. resort columns, fill null values of rain and forward fill missing values in general.
    merged = merged.drop_duplicates()
    merged = merged[['timestamp','temp','dew point','humidity','precip mm','msl','clouds','visibility','wind speed','wind gust','wind dir','snow']]
    merged['precip mm'] = merged['precip mm'].fillna(0)
    merged = merged.fillna(method='pad')
    
    #return the completed DataFrame.
    return merged

In [3]:
#assign variable name to completed dataframe from list of files
merged = merge_csv(os.listdir('.'))

In [203]:
def update_db(merged):
    '''
    Takes in DataFrame merged from csv files, and inserts them to db.
    
    Parameters: 
        Pandas DataFrame containing:
        Int64Index: 106656 entries, 97897 to 19007
        Data columns (total 12 columns):
         #   Column      Non-Null Count   Dtype         
        ---  ------      --------------   -----         
         0   timestamp   106656 non-null  datetime64[ns]
         1   temp        106656 non-null  float64       
         2   dew point   106656 non-null  float64       
         3   humidity    106656 non-null  float64       
         4   precip mm   106656 non-null  float64       
         5   msl         106656 non-null  float64       
         6   clouds      106656 non-null  float64       
         7   visibility  106656 non-null  float64       
         8   wind speed  106656 non-null  float64       
         9   wind gust   106656 non-null  float64       
         10  wind dir    106656 non-null  float64       
         11  snow        106656 non-null  int64         
        dtypes: datetime64[ns](1), float64(10), int64(1)
        
    Returns:
        None
    '''
    #connect to db, assign variable to cursor method
    con.connect()
    insert_main = con.cursor()
    
    #cycle through all values and insert via stored procedure
    for record,temp,dew,humid,precip, msl,clouds,visible,wind_speed,wind_gust,wind_dir,snow in merged.values:
        insert_main.callproc('vantaa_weather_insert',(str(record),temp,dew,humid,precip, msl,clouds,visible,wind_speed,wind_gust,wind_dir,snow))
    
    #commit the db insertion
    con.commit()

In [204]:
update_db(merged)